# Despliegue entorno Hadoop

## Big Data Aplicado

## David Carlón Cembranos

El objetivo de esta tarea es que despliegues el entorno de Hadoop que vamos a usar en el resto de unidades. Para ello debes realizar paso a paso el proceso que se explicó en la guía práctica. Después podrás entregar esta plantilla sustituyendo las celdas de markdown o de código por lo que se pide. Lee detenidamente el enunciado ya que a veces se pedirá que incluyas una imagen, otras texto y en algunas código ejecutable. El libro de Jupyter que entregues deberá tener las celdas completamente ejecutadas en orden secuencial, no debe tener errores y corresponderá con la ejecución real que obtuviste al ejecutarlo en el contenedor `namenode` de tu equipo.

Comienza copiando este libro, `Tarea\ para\ BDA01.ipynb`, en el servidor de Jupyter. Recuerda que el directorio `notebooks` del anfitrión está montado en el directorio `/media/notebooks` del contenedor llamado `namenode` de `docker`. Cualquier cosa que sitúes en este directorio (o en un subdirectorio) será accesible tanto por el anfitrión como por el contenedor.

Si el anterior párrafo no tiene sentido para ti, probablemente debas revisar de nuevo la guía práctica antes de seguir con la tarea práctica.

Es posible que al intentar copiar el libro `Tarea\ para\ BDA01.ipynb` tengas un problema de permisos. Si esto sucede es porque el contenedor de `docker` está asignando la propiedad de los archivos al usuario `root`. Este problema se puede solucionar de distintas formas:

* Asignando permisos de escritura a todos los usuarios (comando `chmod` en Linux).
* Usando el comando `sudo`.
* Copiando el archivo desde el anfitrión al contenedor con el comando `cp` de `docker`:
```bash
docker cp Tarea\ para\ BDA01.ipynb namenode:/media/notebooks/unidad\ 1/
```
* Si todo lo anterior no funciona, pregunta en el foro.

Una vez tengas la plantilla de la tarea en el servidor de Jupyter, comienza a completar las celdas que empiecen por "TODO".

### 1.- Instala `docker` y `docker-compose` en tu equipo y copia una imagen en la que se muestre el resultado de ejecutar en el anfitrión las siguientes instrucciones:

```bash
docker --version
docker-compose --version
``` 

Pantallazo de mi version de docker y docker-compose

![docker version](./img/dockerversion.png)


### 2.- Levanta el entorno de Hadoop con `docker-compose` haciendo lo que se indica en la guía práctica.

El comando que se utiliza para iniciar hadoop es el siguiente:

`docker-compose -f hadoop.yml up`

Pantallazo del inicio de hadoop en el equipo anfitrión.

![docker-compose up](./img/levantarhaddop.png)




### 3.- Muestra en Jupyter el contenido del directorio en el que estás ejecutando este libro.


![jupyter listado](./img/jupyter.png)


### 4.- Realiza todo el proceso que se describe en la guía práctica para obtener el número de palabras que contiene *El Quijote*.

Comando de descarga del quijote dentro de contenedor creando una carpeta:

In [4]:
! wget -P 'libroquijote' https://www.gutenberg.org/files/2000/2000-0.txt

--2023-12-12 17:42:33--  https://www.gutenberg.org/files/2000/2000-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2226045 (2.1M) [text/plain]
Saving to: ‘libroquijote/2000-0.txt’

2000-0.txt          100%[===================>]   2.12M  2.09MB/s    in 1.0s    

2023-12-12 17:42:45 (2.09 MB/s) - ‘libroquijote/2000-0.txt’ saved [2226045/2226045]



In [7]:
with open('libroquijote/2000-0.txt') as f:
    lines = f.readlines()

In [8]:
len(lines)

38062

In [9]:
head = 24
tail = 360

book = lines[head:-tail]
book = ''.join(book)
book = book.lower()

import re

book = re.split('\W+', book)

list(filter(lambda word: len(word) == 0, book))
book = list(filter(lambda word: len(word), book))
book[0:10]
'El Quijote tiene {} palabras aproximadamente.'.format(len(book))

'El Quijote tiene 383640 palabras aproximadamente.'

### 5.- Crea un proceso en Python que muestre las 10 palabras que más se utilizan en *El Quijote* junto con el número de veces que aparecen en el libro.

Los ejercicios anteriores pretendían simplemente que replicaras el trabajo que se realizó en la guía práctica. Este ejercicio tiene como objetivo que empieces a crear tus primeros programas en Python para que te resulten más fáciles las próximas prácticas.

In [11]:
from collections import Counter
import re

# Lee el texto de El Quijote desde el archivo, excluyendo las primeras 24 líneas y las últimas 360 líneas
file_path = 'libroquijote/2000-0.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    quijote_text = ''.join(lines[24:-360])

# Tokeniza el texto en palabras
words = re.findall(r'\b\w+\b', quijote_text.lower())  # Convierte a minúsculas y tokeniza

# Cuenta la frecuencia de cada palabra
word_counts = Counter(words)

# Obtiene las 10 palabras más comunes
top_words = dict(sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:10])

# Imprime las 10 palabras más comunes junto con su frecuencia
for word, count in top_words.items():
    print(f'{word}: {count}')


que: 20769
de: 18410
y: 18272
la: 10492
a: 9876
en: 8285
el: 8265
no: 6346
los: 4769
se: 4752


### 6.- (OPCIONAL) Los dos últimos ejercicios son tareas habituales en el mundo de Big Data, concretamente pertenecen a un área denominada Procesamiento de Lenguaje Natural (NLP). Sin embargo, la solución que se pidió es trivial y tiene bastantes problemas para ser aplicada en un entorno profesional:

* Por un lado no admite trabajar con un gran volumen de datos. De esto nos ocuparemos en próximas prácticas ya que este es el problema esencial al que tratamos de dar solución en este módulo.
* Por otro, tanto la división en palabras (ejercicio 4), como el cálculo de la frecuencia de aparición (ejercicio 5), hacen suposiciones poco realistas: Para la división en palabras hemos supuesto que cualquier carácter no alfanumérico sirve para separar dos palabras, pero esto no es siempre cierto. Por ejemplo, ¿cuántas palabras hay en O.T.A.N.?, ¿cinco o una? Además, para contar las palabras más frecuentes no hemos tenido en cuenta que una misma palabra se puede presentar en diferentes formas. Por ejemplo: "la" y "las", ¿son la misma palabra o dos palabras diferentes?, ¿y "fue" e "irá"? La respuesta a esta pregunta no es unívoca y dependerá del objetivo del estudio.

Las herramientas de NLP nos permiten tanto dividir un texto en palabras ("tokenization"), como 
obtener el [lema](https://es.wikipedia.org/wiki/Lema_(ling%C3%BC%C3%ADstica)#:~:text=El%20lema%20es%20la%20unidad,o%20no%2C%20de%20un%20lema.) de una palabra ("lemmatisation"). En este ejercicio se pretende que resuelvas los ejercicios 4 y 5 usando [`spaCy`](https://spacy.io/), que es una herramienta NLP popular en el mundo Python. Dado que se trata de una actividad voluntaria y para hacerlo más interesante, no se va a dar ninguna indicación de cómo realizar el proceso. Incluso aunque no llegues a resolver el ejercicio, es interesante que consultes la solución cuando esta se publique.

In [1]:
!pip install spacy

In [2]:
!python3 -m spacy download es_core_news_sm

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [16]:
import spacy
from collections import Counter

# Cargar el modelo de idioma en español de spaCy
nlp = spacy.load('es_core_news_sm')

# Establecer un nuevo límite de longitud
nlp.max_length = 3000000  # Ajusta este valor según sea necesario

# Leer el texto de El Quijote desde el archivo, excluyendo las primeras 24 líneas y las últimas 360 líneas
file_path = 'libroquijote/2000-0.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    quijote_text = ''.join(lines[24:-360])

# Contar las líneas en El Quijote
total_lines = len(lines)
print(f'Total de líneas en El Quijote: {total_lines}')






Total de líneas en El Quijote: 38062


In [17]:
# Procesar el texto con spaCy
doc = nlp(quijote_text)

# Obtener todas las palabras (sin filtrar)
all_words = [token.text.lower() for token in doc if token.is_alpha]



In [18]:
# Cuenta la frecuencia de todas las palabras
word_counts = Counter(all_words)

# Obtiene las 10 palabras más comunes
top_words = dict(word_counts.most_common(10))



In [19]:
# Imprime las 10 palabras más comunes junto con su frecuencia
print("\nPalabras más comunes:")
for word, count in top_words.items():
    print(f'{word}: {count}')

# Imprime el total de palabras en El Quijote
total_words = len(all_words)
print(f'\nTotal de palabras en El Quijote: {total_words}')


Palabras más comunes:
que: 20769
de: 18409
y: 18272
la: 10492
a: 9875
en: 8283
el: 8265
no: 6346
los: 4769
se: 4752

Total de palabras en El Quijote: 383495
